# EdX Dataset - Data wrangling

In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sb

## EdX Dataset
The dataset contains data about particpants who enrolled in MITx and HarvardX courses on EdX platform (Academic Year 2013: Fall 2012, Spring 2013, and Summer 2013). The data includes aggregate records of participants activities on EdX (which some information such as 'user name' de-identified). The dataset has been downloaded from

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/26147&version=10.0

In [2]:
# PATH to data file
file='HMXPC13_DI_v2_5-14-14.csv'
path='/Users/suka/Downloads/dataverse_files'
filename=path+'/'+file

# Description of the DataSet  

* **course_id** unique identifier with information about institution , course name and semester
* **userid_DI:** identifies  user / course participant 
* **registered** registered for course, (=1 for all records ) 
* **viewed:**  indicates the user has viewed the course content 
* **explored** indicates user has  accessed at least half of the chapters in the courseware 
* **certified** indicates the user has completed the course with passing grade 
* **final_cc_cname_DI** indicates the country / region of the user
* **LoE** level of education of the user
* **YoB** user-provided, year of birth. 
* **gender** user-provided. Possible values: m (male), f (female) and o (other). 
* **grade** final grade of the user  i
* **start_time_DI**  date of course registration.
* **last_event_DI** date of last interaction with course,
* **nevents** number of interactions with the course,
* **ndays_act** number of unique days student interacted with course.
* **nplay_video** number of play video events within the course. Example: “52”. 
* **nchapters**  number of chapters  completed by the student 
* **inconsistent_flag** identifies records that are internally inconsistent. 



In [3]:
#  READ THE CSV to data frame 
full_df = pd.read_csv(filename,parse_dates=True)
len(full_df)

641138

# Cleanup
Before we go any further, we need to convert/clean the dataset . Here are the changes done
* drop the dataset which user specified information ( YoB,Education,Gender ) incomplete
* split the course_id to retrieve institution , course name and semester / year
* map the course-id/code to course title (eg 'CS50x':'Computer Science')
* rename the column headings appropriately
* fill NaN with 0 values
* Insert "Age" column
* Set the datatypes of the columns appropriately
* add categorical order for the education level 
* convert the grade(0 to 1) to %

In [4]:

# Extract relevant fields 
data = full_df[['course_id','userid_DI','final_cc_cname_DI','LoE_DI','YoB','gender','start_time_DI','grade','viewed','explored','nevents','ndays_act','nplay_video','nchapters','certified','registered','incomplete_flag']].copy()

# split the course id into multiple fields
a,b,c = data['course_id'].str.split('/').str
data.insert(1,'institution',a)
data.insert(2,'course',b)

# clean up the year which has the _spring,_fall suffixes 
a = c.str.split('_').str[0]
b = c.str.split('_').str[1]

data.insert(3,'year',a)
data.insert(4,'term',b)

data.drop('course_id',axis=1,inplace=True)

# Create the mapping of course-id to title 
courselist = {'CB22x':'Greek Heros', 'CS50x':'Computer Science','ER22x':'Justice','PH207x':'Health Stat','PH278x':'Health Env','14.73x':'Poverty','2.01x':'Structures','3.091x':'SS Chemistry','6.002x':'Circuits','6.00x':'Computer Pgming','7.00x':'Biology','8.02x':'Electricity & Magnetism','8.MReV':'Mechanics'}
data['course'].replace(courselist, inplace=True)

#rename columns 
data.rename(columns={'nchapters':'chapters viewed','ndays_act':'days active','nplay_video':'videos played','course_id': 'course', 'final_cc_cname_DI': 'country','LoE_DI':'education','userid_DI':'user',"start_time_DI":"start-time"}, inplace=True)


data['YoB'] = data.groupby('course')['YoB'].transform(lambda x: x.fillna(x.median()))
data['gender'] = data.groupby('course')['gender'].transform(lambda x: x.fillna(x.value_counts().index[0]))
data['education'] = data.groupby('course')['education'].transform(lambda x: x.fillna(x.value_counts().index[0]))



In [18]:
data.head()

,institution,course,year,term,user,country,education,YoB,gender,start-time,grade,viewed,explored,nevents,days active,videos played,chapters viewed,certified,registered,incomplete_flag
0,HarvardX,Greek Heros,2013,Spring,MHxPC130442623,United States,Bachelor's,1986.0,m,2012-12-19,0,0,0,NaN,9.0,NaN,NaN,0,1,1.0
1,HarvardX,Computer Science,2012,NaN,MHxPC130442623,United States,Bachelor's,1987.0,m,2012-10-15,0,1,0,NaN,9.0,NaN,1.0,0,1,1.0
2,HarvardX,Greek Heros,2013,Spring,MHxPC130275857,United States,Bachelor's,1986.0,m,2013-02-08,0,0,0,NaN,16.0,NaN,NaN,0,1,1.0
3,HarvardX,Computer Science,2012,NaN,MHxPC130275857,United States,Bachelor's,1987.0,m,2012-09-17,0,0,0,NaN,16.0,NaN,NaN,0,1,1.0
4,HarvardX,Justice,2013,Spring,MHxPC130275857,United States,Bachelor's,1987.0,m,2012-12-19,0,0,0,NaN,16.0,NaN,NaN,0,1,1.0


In [5]:

# change the datatypes of some of the columns
data['institution']= data.institution.astype('category')
data['course']= data.course.astype('category')

e_order = ["Less than Secondary","Secondary","Bachelor's","Master's","Doctorate"]
data['education']= data.education.astype('category').cat.set_categories(e_order, ordered=True)

data['grade'] = pd.to_numeric(data['grade'],errors='coerce')
data['grade'] = data['grade'].fillna(0).multiply(100)


data['chapters viewed'].fillna(0,inplace=True) # replace NaN with 0 
data['nevents'].fillna(0,inplace=True) # replace NaN with 0 
data['videos played'].fillna(0,inplace=True) # replace NaN with 0 
data['chapters viewed'].fillna(0,inplace=True) # replace NaN with 0 
data['days active'].fillna(0,inplace=True) # replace NaN with 0 
data['term'].fillna('Fall',inplace=True)

data['gender']= data.gender.astype('category')
data['year']= data.year.astype('int')
data['YoB']= data.YoB.astype('int')
data['nevents']= data.nevents.astype('int')


data['start-time'] = pd.to_datetime(data['start-time'])

# Add a column "age" using YoB (age calculated as on 2014)
data.insert(3,"age", [(x.year-x.YoB) for index, x in data.iterrows() ])

In [12]:
print(data.head(5))

  institution            course  year  age    term            user  \
0    HarvardX       Greek Heros  2013   27  Spring  MHxPC130442623   
1    HarvardX  Computer Science  2012   25    Fall  MHxPC130442623   
2    HarvardX       Greek Heros  2013   27  Spring  MHxPC130275857   
3    HarvardX  Computer Science  2012   25    Fall  MHxPC130275857   
4    HarvardX           Justice  2013   26  Spring  MHxPC130275857   

         country   education   YoB gender       ...        grade  viewed  \
0  United States  Bachelor's  1986      m       ...          0.0       0   
1  United States  Bachelor's  1987      m       ...          0.0       1   
2  United States  Bachelor's  1986      m       ...          0.0       0   
3  United States  Bachelor's  1987      m       ...          0.0       0   
4  United States  Bachelor's  1987      m       ...          0.0       0   

   explored  nevents  days active  videos played  chapters viewed  certified  \
0         0        0          9.0         

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641138 entries, 0 to 641137
Data columns (total 21 columns):
institution        641138 non-null category
course             641138 non-null category
year               641138 non-null int64
age                641138 non-null int64
term               641138 non-null object
user               641138 non-null object
country            641138 non-null object
education          641138 non-null category
YoB                641138 non-null int64
gender             641138 non-null category
start-time         641138 non-null datetime64[ns]
grade              641138 non-null float64
viewed             641138 non-null int64
explored           641138 non-null int64
nevents            641138 non-null int64
days active        641138 non-null float64
videos played      641138 non-null float64
chapters viewed    641138 non-null float64
certified          641138 non-null int64
registered         641138 non-null int64
incomplete_flag    100161 non-null fl

In [7]:
print("Extracted "+str(len(data))+" clean records from "+str(len(full_df))+" in the originsl dataset")
print("No of certifications  in full dataset - "+str(len(full_df[full_df.certified == 1])))
print("No of certifications  in cleaned dataset - "+str(len(data[data.certified == 1])))

Extracted 641138 clean records from 641138 in the originsl dataset
No of certifications  in full dataset - 17687
No of certifications  in cleaned dataset - 17687
